# Preparação dos dados para o Modelo.

In [1]:
import pandas as pd
import numpy as np

# -----------------------------
# 1. Threshold dictionary
# -----------------------------
RATIO_RULES = {
    "current_ratio":        {"min": 0.5, "mid": 1.5, "max": 2.5, "invert": False},
    "quick_ratio":          {"min": 0.3, "mid": 1.0, "max": 2.0, "invert": False},
    "debt_to_equity":       {"min": 0.5, "mid": 1.5, "max": 3.0, "invert": True},
    "debt_to_assets":       {"min": 0.2, "mid": 0.5, "max": 0.8, "invert": True},
    "roe":                  {"min": 0.0, "mid": 0.1, "max": 0.2, "invert": False},
    "roa":                  {"min": 0.0, "mid": 0.05, "max": 0.1, "invert": False},
    "net_margin":           {"min": 0.0, "mid": 0.05, "max": 0.15, "invert": False},
    "interest_coverage":    {"min": 1.0, "mid": 3.0, "max": 5.0, "invert": False},
    "ocf_to_debt":          {"min": 0.1, "mid": 0.3, "max": 0.5, "invert": False},
    "fcf_to_sales":         {"min": 0.0, "mid": 0.05, "max": 0.1, "invert": False},
    "asset_turnover":       {"min": 0.3, "mid": 1.0, "max": 2.0, "invert": False},
    "inventory_turnover":   {"min": 2.0, "mid": 6.0, "max": 10.0, "invert": False},
    "receivable_turnover":  {"min": 3.0, "mid": 8.0, "max": 12.0, "invert": False},
    "sales_growth":         {"min": 0.0, "mid": 0.1, "max": 0.2, "invert": False},
    "net_income_growth":    {"min": 0.0, "mid": 0.1, "max": 0.2, "invert": False},
    "retained_to_assets":   {"min": 0.0, "mid": 0.15, "max": 0.3, "invert": False},
    "fx_position":          {"min": -1.0, "mid": 0.0, "max": 1.0, "invert": False},
}

# -----------------------------
# 2. Normalization function (0–10)
# -----------------------------
def score_ratio(x, rule):
    """Maps a numeric ratio to a 0–10 scale based on thresholds."""
    if pd.isna(x): 
        return np.nan
    minv, midv, maxv = rule["min"], rule["mid"], rule["max"]
    invert = rule["invert"]

    if invert:
        # lower is better
        if x <= minv: return 10
        elif x >= maxv: return 0
        else: return 10 * (maxv - x) / (maxv - minv)
    else:
        if x <= minv: return 0
        elif x >= maxv: return 10
        else:
            if x <= midv:
                return 5 * (x - minv) / (midv - minv)
            else:
                return 5 + 5 * (x - midv) / (maxv - midv)

# -----------------------------
# 3. Compute scores
# -----------------------------
def compute_financial_health(df, weights=None):
    """Calculates 0–10 scores and weighted composite index."""
    df_scores = df.copy()
    for col, rule in RATIO_RULES.items():
        if col in df_scores.columns:
            df_scores[col + "_score"] = df_scores[col].apply(score_ratio, rule=rule)

    # Default equal weighting if none provided
    if weights is None:
        weights = {cat: 1 for cat in RATIO_RULES.keys()}

    total_weight = sum(weights.values())
    score_cols = [c for c in df_scores.columns if c.endswith("_score")]

    df_scores["Financial_Health_Index"] = sum(
        df_scores[c] * weights[c.replace("_score", "")] for c in score_cols
    ) / total_weight

    return df_scores

In [3]:
df=pd.read_csv('C:/Users/gregorio/TCC-CD-USP/archives/df_wide.csv')
df.head()

,CompanyCode,Ano,Investimentos em Capital Fixo,(Subtotal),Ajuste de Capital de Participação Mútua (-),Ajustes:,Amortização e Quotas de Exaustão,Ativo Fiscal Diferido,Ativos Circulantes,Ativos Imobilizados,...,Receitas de Atividades de Investimento,Reservas Restritas Separadas do Lucro,Reservas de Capital de Emissão de Ações,TOTAL DE ATIVOS,TOTAL DO PASSIVO E PATRIMÔNICO LÍQUIDO,Valores de Caixa no Início do Período,Variação nas Dívidas Financeiras,Variação no Caixa e Equivalentes,Vendas Domésticas,Vendas Internacionais
0,ADEL,2008,NaN,27018810.5,0.0,NaN,NaN,0.0,45582654.0,14716940.0,...,NaN,2957664.0,0.0,60753401.0,60753401.0,NaN,NaN,NaN,87260709.0,4865016.0
1,ADEL,2009,NaN,27018810.5,0.0,NaN,NaN,0.0,45582654.0,14716940.0,...,NaN,2957664.0,0.0,60753401.0,60753401.0,NaN,NaN,NaN,87260709.0,4865016.0
2,ADEL,2010,NaN,33872750.5,0.0,NaN,NaN,0.0,58621451.0,13861696.0,...,NaN,4729678.0,0.0,73046265.0,73046265.0,NaN,NaN,NaN,104427004.0,5399918.0
3,ADEL,2011,NaN,43914250.0,0.0,NaN,NaN,113998.0,75550839.0,14762743.0,...,NaN,5450104.0,0.0,91031840.0,91031840.0,NaN,NaN,NaN,132355741.0,6157158.0
4,ADEL,2012,NaN,54610053.0,0.0,NaN,NaN,346418.0,93484162.0,16136254.0,...,NaN,6378203.0,0.0,111340127.0,111340127.0,NaN,NaN,NaN,162911678.0,7611179.0
